# Planetary Training

OBS: This notebook is for running in the MSPC only!!!!

In [2]:
%load_ext autoreload
%autoreload 2

import importlib    

# check if it is using CPU or GPU
# First the CPU version
if importlib.util.find_spec('torch') is None:
    %pip install --quiet torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu
    
if importlib.util.find_spec('fastai') is None:
    # Then install fastai from code
    !cd /tmp; git clone https://github.com/fastai/fastai -q
    %pip -q install -e /tmp/fastai 
    
# Install WaterNet
if importlib.util.find_spec('waternet') is None:
    %pip install -e /home/jovyan/projects/WaterNet/
    
from waternet.wndataset import WNDataSet
from fastai.vision.all import *



### Creating the Datasets

In [3]:
from waternet.wnbaseimage import WNBaseImage
from waternet.wnstacimage import WNStacImage
from waternet.wnsegmantationitem import WNSegmentationItem
from pathlib import Path

In [18]:
masks_path = '/home/jovyan/masks/france'
training_bands = ['B04', 'B03', 'B02', 'B08', 'B11', 'B12']
shape=(5490, 5490)
patch_size=(512, 512)
step=262

# masks_path='d:/temp/22KEV/'

In [19]:
# first, let's load the Bordeaux mask manually
mask = WNBaseImage(Path(masks_path)/'T30TXQ_20180911_Bordeaux_summer.tif', shape=shape)

In [20]:
# now, let's load the image
img = WNStacImage.from_tile(tile='30TXQ', str_date='2018-09-11', shape=shape)

In [21]:
segitem = WNSegmentationItem(img=img, mask=mask, patch_size=patch_size, step=step)

In [22]:
train_ds = WNDataSet(items=[segitem])
train_ds

WNDataset instance with 1 images
Loaded: 0 items
Empty: 1 items
Loading: 0 items

### Creating the Validation Dataset

In [23]:
# first, let's load the Bordeaux mask manually
valid_mask = WNBaseImage(Path(masks_path)/'T30TXQ_20190223_Bordeaux_winter.tif', shape=shape)

In [24]:
# now, let's load the image
valid_img = WNStacImage.from_tile(tile='30TXQ', str_date='2019-02-23', shape=shape)

In [25]:
valid_segitem = WNSegmentationItem(img=valid_img, mask=valid_mask, patch_size=patch_size, step=step)

In [26]:
valid_ds = WNDataSet(items=[valid_segitem])
valid_ds

WNDataset instance with 1 images
Loaded: 0 items
Empty: 1 items
Loading: 0 items

### Creating the DataLoader

In [27]:
train_ds.bands = training_bands
valid_ds.bands = training_bands

### Creating Dataloaders

In [28]:
# Create two dataloaders (for training and validation)
# Put it in the GPU
dls = DataLoaders(
    DataLoader(train_ds, bs=8),
    DataLoader(valid_ds, bs=8)
).cuda()

In [29]:
# m, t = dls.valid.one_batch()
# m.device

In [30]:
# dls.train.one_batch()[0].shape

### Creating the Learner

In [32]:
learner = unet_learner(
    dls, arch=resnet18, pretrained=True, normalize=False, n_in=6, n_out=3, loss_func=CrossEntropyLossFlat(axis=1)
)

/srv/conda/envs/notebook/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [33]:
# learner.load('Resnet18-6chnls-France-2')

In [34]:
cuda_model = learner.cuda()

### Checking model and predictions

In [35]:
learner.unfreeze()

In [20]:
# learner.summary()

In [21]:
# batch, target = dls.train.one_batch()
# print(batch.shape, target.shape)

In [22]:
# Calculate the predictions for the 4 patches (1 batch)
# pred = learner.model(batch)
# pred.shape

In [23]:
# CrossEntropyLossFlat(axis=1)(pred, target)

### Running LrFind

Up to now, evertying was done in the CPU, but to run large amounts, it is necessary to load the data in the Cuda device. For that we will put them in the GPU and find the best learning rate.

In [43]:
# learner.lr_find()

In [38]:
learner.fit_one_cycle(10, lr_max=slice(1e-6, 5e-4), wd=0.01)

epoch,train_loss,valid_loss,time
0,0.482301,0.310339,01:38
1,0.558546,0.259914,01:32
2,0.755606,0.388923,01:32
3,0.492829,0.141602,01:33
4,0.266383,0.184377,01:32
5,0.152248,0.529636,01:32
6,0.104152,0.578665,01:32
7,0.074611,0.204482,01:32
8,0.060874,0.264799,01:32
9,0.048816,0.253057,01:32


In [39]:
learner.save('Resnet18-6chnls-France-1')

Path('models/Resnet18-6chnls-France-1.pth')

In [40]:
learner.fit_one_cycle(10, lr_max=slice(1e-6, 1e-5), wd=0.02)

epoch,train_loss,valid_loss,time
0,0.041191,0.254853,01:32
1,0.041116,0.264444,01:32
2,0.040149,0.276989,01:33
3,0.038090,0.286266,01:32
4,0.035826,0.291641,01:32
5,0.033704,0.293217,01:32
6,0.031883,0.292615,01:32
7,0.030488,0.291750,01:32
8,0.029586,0.291421,01:32
9,0.029119,0.291379,01:33


In [41]:
learner.save('Resnet18-6chnls-France-2')

Path('models/Resnet18-6chnls-France-2.pth')

In [44]:
learner.fit_one_cycle(10, lr_max=slice(1e-5, 1e-4), wd=0.02)

epoch,train_loss,valid_loss,time
0,0.028856,0.312829,01:28
1,0.029456,1.315528,01:31
2,0.030989,1.008907,01:32
3,0.031287,0.999674,01:32
4,0.031643,0.918555,01:32
5,0.027432,0.785387,01:32
6,0.023587,0.486750,01:33
7,0.022026,0.462193,01:33
8,0.017639,0.446406,01:33
9,0.015429,0.445930,01:32


In [45]:
learner.save('Resnet18-6chnls-France-3')

Path('models/Resnet18-6chnls-France-3.pth')

In [ ]:
learner.model_dir

### Como ver os resultados???

In [ ]:
from waternet.wnvisualizer import WNVisualizer

In [ ]:
learner = unet_learner(
    dls, arch=resnet18, pretrained=False, normalize=False, n_in=6, n_out=3, loss_func=CrossEntropyLossFlat(axis=1)
)
learner.load('Resnet18-6chnls-untrained')
model = learner.cpu()

In [ ]:
visu = WNVisualizer(dls.valid, learner)

In [ ]:
visu.predict_item(0)['item'].shape

In [ ]:
visu.show_pred(800)

In [ ]:
dls.valid.dataset.loaded_status